# Simulation Tutorial
### Lisa DeBruine
### 2020-02-17
## Setup 

### Julia

Load the packages we'll be using in Julia

In [ ]:
using MixedModels        # run mixed models
using MixedModelsSim     # simulation functions for mixed models
using RCall              # call R functions from inside Julia
using DataFrames, Tables # work with data tables
using Random, Statistics # statistical functions
using CSV                # write CSV files

### R

Also load any packages we'll be using in R through `RCall()`.

In [ ]:
R"""
library(ggplot2) # for visualisation
library(dplyr)   # for data wrangling
library(tidyr)   # for data wrangling
""";

### Define Custom functions

It's useful to be able to weave your file quickly while you're debugging, 
so set the number of simulations to a relatively low number while you're 
setting up your script and change it to a larger number when everything
is debugged.

In [ ]:
nsims = 1000 # set to a low number for test, high for production

In [ ]:
# hopefully in MixedModelsSim soon

function sim_to_df(sims)
    tab = DataFrame()
    for (i, sim) in enumerate(sims)
        df = DataFrame(sim)
        df[!, :iteration] .= i
        df[!, :var] .= string.(collect(keys(sim)))
        append!(tab, df)
    end
    longtab = stack(tab, 1:(ncol(tab)-2), variable_name = :coefname)
    widetab = unstack(longtab, :var, :value)
    rename!(widetab, ["coefname", "iteration",  "p",  "se",  "z",  "beta" ])
    sort!(widetab, [:iteration])
    select!(widetab, [:iteration, :coefname, :beta, :se, :z, :p])
end

function power_table(sim, alpha = 0.05)
    pvals = DataFrame(columntable(sim).p)
    pvals = stack(pvals) 
    pwr = by(pvals, :variable, :value => x->mean(x.<alpha) )
    rename!(pwr, ["coefname", "power"])
end

function simdat_crossed(subj_n = 1, item_n = 1;
    subj_btwn = nothing, item_btwn = nothing, both_win = nothing)

    # set up subj table
    if isnothing(subj_btwn)
        subj_vals = [nlevels(subj_n, "S")]
        sb_vars = []
    else
        sc = values(subj_btwn) |> collect
        sc = vcat([nlevels(subj_n)], sc)
        subj_prod = Iterators.product(sc...)
        subj_total_n = length(subj_prod)
        subj_vals = columntable(subj_prod) |> collect
        subj_vals[1] = nlevels(subj_total_n, "S")
        sb_vars = collect(keys(subj_btwn))
    end

    subj_names = vcat(["subj"], sb_vars)
    subj = NamedTuple{Tuple(Symbol.(subj_names))}(subj_vals)

    # set up item table
    if isnothing(item_btwn)
        item_vals = [nlevels(item_n, "I")]
        ib_vars = []
    else
        ic = values(item_btwn) |> collect
        ic = vcat([nlevels(item_n, "I")], ic)
        item_prod = Iterators.product(ic...)
        item_total_n = length(item_prod)
        item_vals = columntable(item_prod) |> collect
        item_vals[1] = nlevels(item_total_n, "I")
        ib_vars = collect(keys(item_btwn))
    end

    item_names = vcat(["item"], ib_vars)
    item = NamedTuple{Tuple(Symbol.(item_names))}(item_vals)

    # set up within both table
    if (isnothing(both_win))
        # cross the subject and item tables 
        design = factorproduct(subj, item) |> DataFrame
    else 
        wc = values(both_win) |> collect
        win_prod = Iterators.product(wc...)
        win_vals = columntable(win_prod) |> collect
        win_names = collect(keys(both_win))
        win = NamedTuple{Tuple(Symbol.(win_names))}(win_vals)

        # cross the subject and item tables with any within factors 
        design = factorproduct(subj, item, win) |> DataFrame
    end

    # add random numbers as a DV
    design.dv = randn(nrow(design))

    design

end

#### Define: ggplot_betas

This function plots the beta values returned from `simulate_waldtests` using ggplot in R.
If you set a figname, it will save the plot to the specified file.

In [ ]:
function ggplot_betas(sim, figname = 0, width = 7, height = 5) 

    beta_df = DataFrame(columntable(sim).β)

    R"""
        p <- $beta_df %>%
            gather(var, val, 1:ncol(.)) %>%
            ggplot(aes(val, color = var)) +
            geom_density(show.legend = FALSE) +
            facet_wrap(~var, scales = "free")

        if (is.character($figname)) {
            ggsave($figname, p, width = $width, height = $height)
        }

        p
    """
end

## Existing Data

Load existing data from this morning's tutorial. Set the contrasts and run model 4 from the tutorial.

In [ ]:
# load data
kb07 = MixedModels.dataset("kb07");

# set contrasts
contrasts = Dict(:spkr => HelmertCoding(), 
                 :prec => HelmertCoding(), 
                 :load => HelmertCoding());

# define formula
kb07_f = @formula( rt_trunc ~ 1 + spkr+prec+load + (1|subj) + (1+prec|item) );

# fit model
kb07_m = fit(MixedModel, kb07_f, kb07, contrasts=contrasts)

### Simulate data with same parameters

Use the `simulate_waldtests()` function to run `j nsims` iterations of data sampled 
using the parameters from `m4`. Set up a random seed to make the simulation reproducible. 
You can use your favourite number.

In [ ]:
# seed for reproducibility
rng = MersenneTwister(8675309);

# run 1000 iterations
kb07_sim = simulate_waldtests(rng, nsims, kb07_m, use_threads = true);

Save all data to a csv file.

In [ ]:
kb07_sim_df = sim_to_df(kb07_sim)

CSV.write("sim/kb07_sim.csv", kb07_sim_df)

Plot betas in ggplot.

In [ ]:
ggplot_betas(kb07_sim, "fig/kb07_betas.png");

![](fig/m4_betas.png)


### Power calculation

In [ ]:
power_table(kb07_sim)

### Change parameters

Let's say we want to check our power to detect effects of spkr, prec, and load 
that are half the size of our pilot data. We can set a new vector of beta values 
with the `β` argument to `simulate_waldtests`.

In [ ]:
newβ = kb07_m.β
newβ[2:4] = kb07_m.β[2:4]/2

kb07_sim_half = simulate_waldtests(rng, nsims, kb07_m, β = newβ, use_threads = true);

power_table(kb07_sim_half)

# Simulating Data from Scratch


## simdat_crossed

We will set a design where `subj_n` subjects per `age` group (O or Y) respond to `item_n` items in each of two `condition`s (A or B). Create a simulated data structure with `simdat(sub_n, item_n)`.


Running the function with the defaults gives you the general experimental design structure
(i.e., 1 subject and 1 stimulus in each cell).

In [ ]:
# put between-subject factors in a Dict
subj_btwn = Dict("age" => ["O", "Y"])

# put within-subject/item factors in a Dict
both_win = Dict("condition" => ["A", "B"])

# simulate data
dat = simdat_crossed(10, 30, subj_btwn = subj_btwn, both_win = both_win )

```

## Set up model

In [ ]:
# set contrasts
contrasts = Dict(:age => HelmertCoding(), 
                 :condition => HelmertCoding());

f1 = @formula dv ~ 1 + age * condition + (1|item) + (1|subj);
m1 = fit(MixedModel, f1, dat, contrasts=contrasts)

Because the `dv` is just random numbers from N(0,1), there will be basically no subject or item random variance, residual variance will be near 1.0, and the estimates for all effects should be small. Now we can specify fixed and random effects directly in `simulate_waldtests`. 

## Simulate

* Set a seed for reproducibility
* specify new β, σ, and θ

In [ ]:
rng = MersenneTwister(8675309);

new_beta = [0, 0.25, 0.25, 0]
new_sigma = 2.0
new_theta = [1.0, 1.0]

sim1 = simulate_waldtests(rng, nsims, m1, 
                        β = new_beta, 
                        σ = new_sigma, 
                        θ = new_theta,
                        use_threads = true);

## Explore simulation output

In [ ]:
ggplot_betas(sim1, "fig/simbetas.png");

![](fig/simbetas.png)


## Power

In [ ]:
power_table(sim1)

## Write a function to vary something

In [ ]:
function mysim(subj_n, item_n, nsims = 1000, 
               beta  = [0, 0, 0, 0],
               sigma = 2.0, 
               theta = [1.0, 1.0],
               seed = convert(Int64, round(rand()*1e8))
               )
    # generate data
    dat = simdat_crossed(subj_n, item_n, subj_btwn = subj_btwn, both_win = both_win )

    # set contrasts
    contrasts = Dict(:age => HelmertCoding(), 
                     :condition => HelmertCoding());

    # set up model
    f = @formula dv ~ 1 + age*condition + (1|item) + (1|subj);
    m = fit(MixedModel, f, dat, contrasts=contrasts)

    # run simulation
    rng = MersenneTwister(seed);

    simulate_waldtests(
        rng, nsims, m, 
        β = beta, 
        σ = sigma, 
        θ = theta, 
        use_threads = true
    );
end

Run simulations over a range of values for any parameter.

In [ ]:
# varying
subj_ns = [20, 30, 40]
item_ns = [10, 20, 30]

# fixed
nsims = 1000
new_beta = [0, 0.4, 0.1, 0]
new_sigma = 2.0
new_theta = [1.0, 1.0]

d = DataFrame()

for subj_n in subj_ns
    for item_n in item_ns
        s = mysim(subj_n, item_n, nsims, new_beta, new_sigma, new_theta);
        pt = power_table(s)
        pt[!, :item_n] .= item_n
        pt[!, :sub_n] .= subj_n
        append!(d, pt)
    end
end

# save the data in long format
CSV.write("sim/power.csv", d)

# spread the table for easier viewing
unstack(d, :coefname, :power)

## Convert this file

In [ ]:
# using Weave

# convert to html
# weave("simulation_tutorial.jmd")

# convert to a python notebook
# convert_doc("simulation_tutorial.jmd", "simulation_tutorial.ipynb")

# convert to md for README
# weave("simulation_tutorial.jmd", doctype="pandoc", out.path = "README.md")